<a href="https://colab.research.google.com/github/leozungalo/leozungalo/blob/main/(L%C3%A9o)CNN_Proptotipo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importando tensorflow, keras e outras bibliotecas auxiliares.

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

# Bibliotecas auxiliares
import numpy as np
import matplotlib.pyplot as plt

# Só pra checar a versão
import matplotlib

print(tf.__version__)
print(np.__version__)
print(matplotlib.__version__)
!python --version

2.15.0
1.25.2
3.7.1
Python 3.10.12


`Importando os Datasets de treino, validação e teste.`

In [ ]:
#Paths de treino, validação e teste
train_path = '/content/dataset3/treino'
valid_path = '/content/dataset3/validacao'
test_path = '/content/dataset3/teste'

#Importando dataset de treino com 1050 imagens com tumor e 1050 sem o tumor. Padronizando o tamanho de todas em 128x128
train_set = tf.keras.preprocessing.image_dataset_from_directory(train_path, labels='inferred', image_size=[128,128])

#Importando dataset de validação com 300 imagens com tumor e 300 sem o tumor. Padronizando o tamanho de todas em 128x128
valid_set = tf.keras.preprocessing.image_dataset_from_directory(valid_path, labels='inferred', image_size=[128,128])

#Importando dataset de teste com 150 imagens com tumor e 150 sem o tumor. Padronizando o tamanho de todas em 128x128
test_set = tf.keras.preprocessing.image_dataset_from_directory(test_path, labels='inferred', image_size=[128,128])

Visualizando 9 imagens de cada dataset.

In [ ]:
#Dataset de treino
plt.figure(figsize=(10, 10))

for images, labels in train_set.take(1):
  for i in range(7):
    ax = plt.subplot(3,3,i+1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title("Negativo" if int(labels[i]) == 1 else "Positivo")
    plt.axis('off')

In [ ]:
#Dataset de validação
plt.figure(figsize=(10, 10))

for images, labels in valid_set.take(1):
  for i in range(2):
    ax = plt.subplot(3,3,i+1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title("Negativo" if int(labels[i]) == 1 else "Positivo")
    plt.axis('off')

In [ ]:
#Dataset de teste
plt.figure(figsize=(10, 10))

for images, labels in test_set.take(1):
  for i in range(1):
    ax = plt.subplot(3,3,i+1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title("Negativo" if int(labels[i]) == 1 else "Positivo")
    plt.axis('off')

Modelando rede

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.Conv2D(4, activation='relu', kernel_size=5),
    tf.keras.layers.Conv2D(32, activation='relu', kernel_size=5),
    tf.keras.layers.Conv2D(32, activation='relu', kernel_size=5),
    tf.keras.layers.Conv2D(64, activation='relu', kernel_size=5),
    tf.keras.layers.Conv2D(128, activation='relu', kernel_size=5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Preparando configurações de treino, validação e teste

In [ ]:
#Inicializando o dataset de treino
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

#Inicializando o dataset de validação
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

#Inicializando o dataset de teste
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

#Importando dataset de treino
train_generator = train_datagen.flow_from_directory(
    "/content/dataset3/treino",
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

#Importando dataset de validação
val_generator = val_datagen.flow_from_directory(
    "/content/dataset3/validacao",
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

#Importando dataset de teste
test_generator = test_datagen.flow_from_directory(
    "/content/dataset3/teste",
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

Criando modelo e epochs de treino.

In [ ]:
#Preparando modelo para treino. Otimizador mais popular configurado.
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

#Realizando os ciclos de treino
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    verbose=2
)

# Calculando a acurácia do modelo
predictions = model.predict(test_generator)
predicted_labels = np.round(predictions).flatten()
true_labels = test_generator.classes
accuracy = np.mean(predicted_labels == true_labels)
print("Acurácia do modelo:", accuracy)